# Day 07

## Part 1

In [30]:
input_file = 'inputs.txt'
with open(input_file, 'r') as f:
    puzzle_inputs = [line.strip() for line in f]
puzzle_inputs[:10]

['$ cd /',
 '$ ls',
 'dir brhvclj',
 'dir clnvqg',
 'dir dtqtvvrn',
 'dir lcz',
 'dir pcqjncwl',
 'dir qwvfpgl',
 'dir rtmj',
 'dir shg']

In [5]:
import re

file_pattern = re.compile(r'^(\d+)\s(.+)')
dir_pattern = re.compile(r'^dir (.+)')

In [10]:
m = dir_pattern.match('dir ddasf')
m.groups()

('ddasf',)

In [31]:
# Parse the directories
dir_tree = {
}
current_dir = dir_tree
breadcrumbs = []

def cd(to_dir = '/'):
    if to_dir == '/':
        breadcrumbs.clear()
        return dir_tree

    if to_dir == '..':
        # Removes the current level
        breadcrumbs.pop()
    else:
        breadcrumbs.append(to_dir)

    _current_dir = dir_tree
    for dir in breadcrumbs:
        _current_dir = _current_dir[dir]
    return _current_dir

for line in puzzle_inputs:
    if line.startswith('$ cd '):
        current_dir = cd(to_dir=line.replace('$ cd ', ''))
        continue

    if line.startswith('$ ls'):
        # Do we need to care?
        continue

    file_matches = file_pattern.match(line)
    if file_matches:
        file_size, file_name = file_matches.groups()
        current_dir[file_name] = int(file_size)
        continue

    dir_matches = dir_pattern.match(line)
    if dir_matches:
        dir_name = dir_matches.group(1)
        if dir_name not in current_dir:
            current_dir[dir_name] = {}
        continue

dir_tree

{'brhvclj': {'mtlscfrd.gdr': 40016,
  'mvslzl': {'dngldfww': {'wcrvztrh.mzb': 102218},
   'dzplphqw': {'brhvclj': 93724, 'cwqwcjc.lgd': 169467}},
  'npp.vjv': 112449,
  'pbgjwb': 46674,
  'qdtls': {'nnlzrwgh': {'fnb': 141976},
   'svc': {'hzrml': 87249, 'qbmgmmc': 46855, 'zsbsq.cvq': 147235},
   'tcq.lng': 168425,
   'vjrntvbz.sgn': 99608,
   'wshf': {'npp.vjv': 25072},
   'wtmdzgvg': {'cfpfdqq.vfd': 165660,
    'dhbrmn': {'brhvclj.hcf': 200474, 'npp.vjv': 233675, 'ttntt': 218734},
    'dscvmj.wpn': 176203,
    'rcdwft': {'wvfdjv': {'mdrr': 104669}}}},
  'tfns': {'clnvqg': {'brhvclj': {'brhvclj': {'mlrcgl': 246966,
      'ttntt.ztq': 93485}},
    'cdfj': 103188,
    'svc': 256254,
    'tndnh.vgg': 72302},
   'mlft.cvp': 15637,
   'pqp': {'svc': {'bsczwq': 202934}, 'ttntt.qpm': 206837},
   'svc.mqv': 77675,
   'ttntt': {'dfmgqmvm': {'qbms': {'lvfphhpw.sjv': 20046,
      'pdbtbmf': 186516,
      'svc.dqh': 149493,
      'tdvtcmwz': {'hpdm.ctv': 100584}},
     'wczpvsw.ddr': 201736,
     

In [32]:
dir_sizes = {}

def dir_size(directory, dir_sizes, current_path):
    current_dir_size = sum(v for v in directory.values() if not isinstance(v, dict))
    sub_directories = (v for v in directory.values() if isinstance(v, dict))

    if not sub_directories:
        dir_sizes['/'.join(current_path)] = current_dir_size
        return current_dir_size, current_dir_size if current_dir_size <= 100000 else 0

    sum_small_directory_sizes = 0
    for subdir_name, subdir in directory.items():
        if not isinstance(subdir, dict):
            continue
        current_path.append(subdir_name)
        subdir_size, subdir_size_if_small = dir_size(
            directory=subdir,
            dir_sizes=dir_sizes,
            current_path=current_path
        )
        current_path.pop()
        current_dir_size += subdir_size
        sum_small_directory_sizes += subdir_size_if_small

    dir_sizes['/'.join(current_path)] = current_dir_size

    return current_dir_size, current_dir_size + sum_small_directory_sizes if current_dir_size <= 100000 else sum_small_directory_sizes

space, small_dir = dir_size(dir_tree, dir_sizes, current_path=[])
print(small_dir)

1770595


## Part 2

In [33]:
space_to_free = 30000000 - (70000000 - space)
space_to_free

2143088

In [34]:
min_space = 70000000
for value in dir_sizes.values():
    if value >= space_to_free:
        min_space = min(min_space, value)
min_space

2195372